Environment: napari_sparrow

In [ ]:
%load_ext autoreload
%autoreload 2

### Import packages

In [ ]:
import sparrow as sp
import os
import spatialdata as sd
import anndata as ad
import numpy as np
import cv2 as cv2
from skimage import io
import geopandas as gpd
from shapely.geometry import Point
import spatialdata as sd
import pandas as pd
from sparrow.table._table import _back_sdata_table_to_zarr
from matplotlib import pyplot as plt
import seaborn as sns
from skimage.measure import regionprops_table
from matplotlib.colors import LinearSegmentedColormap


### Set paths and create folders

In [ ]:
# Specify path to root folder
root_folder = r'D:\Data\2023-07-CarolineAsselman-FIm\Analysis_v2'

# Specify ROI name
ROI = 'A1_ROI1'

# Path to input folders
sdata_path = os.path.join(root_folder, 'output', ROI, 'downstream_analysis')

# Path to output folders
output_path = os.path.join(root_folder, 'analysis_seurat', ROI)
plots_path = os.path.join(output_path, 'plots_2')
os.makedirs(output_path, exist_ok = True)
os.makedirs(plots_path, exist_ok = True)

### Create or read sdata

In [ ]:
sdata = sd.read_zarr(os.path.join(sdata_path, "sdata.zarr"))
sdata

### Read in seurat clusters

In [ ]:
df = pd.read_csv(os.path.join(output_path, "cells_per_cluster.txt"), delimiter = '\t')
df = df[['cell_id', 'seurat_clusters']]
df['seurat_clusters'] = df['seurat_clusters'].astype(str)
df = df.set_index('cell_id')
df.index = df.index.astype(str)
df

### Add seurat clusters to anndata table

In [ ]:
sdata.table.obs = sdata.table.obs.merge(df, left_index=True, right_index=True, how='outer')

### Create plots

In [ ]:
from matplotlib.colors import ListedColormap

# Define the colors in a dictionary format
colors_dict = {
    '0': "#f0e442",
    '1': "#d55e00",
    '2': "#009e73",
    '3': "#0072b2",
    '4': "#cc79a7",
}

# Create a custom colormap
cmap = ListedColormap([colors_dict[key] for key in sorted(colors_dict.keys())])


In [ ]:
sp.pl.plot_shapes(
    sdata, 
    img_layer='raw_image',
    channel = 'DAPI',
    shapes_layer="segmentation_mask_merged_boundaries", 
    output=os.path.join(plots_path , 'seurat_clusters_all.png'),
    alpha=1,
    cmap=cmap,
    column='seurat_clusters',
    figsize=(17.5, 11.25)
)

In [ ]:
seurat_clusters = ["0", "1", "2", "3", "4"]

for cluster in seurat_clusters:
    
    sdata.table.obs[cluster] = 'Other'
    sdata.table.obs.loc[sdata.table.obs['seurat_clusters']== cluster, cluster] = cluster
    sdata.table.obs.loc[sdata.table.obs['seurat_clusters'].isna(), cluster] = np.nan
    
    cmap_sub = ListedColormap([colors_dict[cluster],'#AEAEAE'])
    
    sp.pl.plot_shapes(
        sdata, 
        img_layer='raw_image',
        channel = 'DAPI',
        shapes_layer="segmentation_mask_merged_boundaries", 
        output=os.path.join(plots_path , f'seurat_clusters_{cluster}.png'),
        alpha=1,
        cmap=cmap_sub,
        column=cluster,
        figsize=(17.5, 11.25)
    )
    
    sdata.table.obs = sdata.table.obs.drop([cluster], axis=1)

In [ ]:
sdata.table.obs['0_3'] = np.nan
sdata.table.obs.loc[sdata.table.obs['seurat_clusters']== '0', '0_3'] = '0'
sdata.table.obs.loc[sdata.table.obs['seurat_clusters']== '1', '0_3'] = 'Other'
sdata.table.obs.loc[sdata.table.obs['seurat_clusters']== '2', '0_3'] = 'Other'
sdata.table.obs.loc[sdata.table.obs['seurat_clusters']== '3', '0_3'] = '3'
sdata.table.obs.loc[sdata.table.obs['seurat_clusters']== '4', '0_3'] = 'Other'

cmap_sub = ListedColormap([colors_dict['0'], colors_dict['3'], '#AEAEAE'])

sp.pl.plot_shapes(
    sdata, 
    img_layer='raw_image',
    channel = 'DAPI',
    shapes_layer="segmentation_mask_merged_boundaries", 
    output=os.path.join(plots_path , 'seurat_clusters_0_3.png'),
    alpha=1,
    cmap=cmap_sub,
    column='0_3',
    figsize=(17.5, 11.25)
)

sdata.table.obs = sdata.table.obs.drop(['0_3'], axis=1)

### Plots for immune cells

In [ ]:
df_immune = pd.read_csv(os.path.join(output_path, "immunecells_per_cluster.txt"), delimiter = '\t')
df_immune = df_immune[['cell', 'seurat_clusters']]
df_immune['seurat_clusters_immune'] = df_immune['seurat_clusters'].astype(str)
df_immune = df_immune.drop('seurat_clusters', axis=1)
df_immune = df_immune.set_index('cell')
df_immune.index = df_immune.index.astype(str)
df_immune

In [ ]:
# Define the colors in a dictionary format
colors_dict = {
    '0': "#d55e00",
    '1': "#0072b2",
    '2': "#f0e442",
    '3': "#cc79a7",
}

# Create a custom colormap
cmap_sub = ListedColormap([colors_dict[key] for key in sorted(colors_dict.keys())])


In [ ]:
sdata.table.obs = sdata.table.obs.merge(df_immune, left_index=True, right_index=True, how='outer')
# sdata.table.obs['seurat_clusters_immune'] = sdata.table.obs['seurat_clusters_immune'].fillna('Other')

In [ ]:
sp.pl.plot_shapes(
    sdata, 
    img_layer='raw_image',
    channel = 'DAPI',
    shapes_layer="segmentation_mask_merged_boundaries", 
    output=os.path.join(plots_path , 'seurat_clusters_immune_all.png'),
    alpha=1,
    cmap=cmap_sub,
    column='seurat_clusters_immune',
    figsize=(17.5, 11.25)
)

In [ ]:
seurat_clusters_immune = ["0", "1", "2", "3"]

for cluster in seurat_clusters_immune:
    
    sdata.table.obs[cluster] = 'Other'
    sdata.table.obs.loc[sdata.table.obs['seurat_clusters_immune']== cluster, cluster] = cluster
    sdata.table.obs.loc[sdata.table.obs['seurat_clusters_immune'].isna(), cluster] = np.nan
    
    cmap_sub = ListedColormap([colors_dict[cluster],'#AEAEAE'])
    
    sp.pl.plot_shapes(
        sdata, 
        img_layer='raw_image',
        channel = 'DAPI',
        shapes_layer="segmentation_mask_merged_boundaries", 
        output=os.path.join(plots_path , f'seurat_clusters_immune_{cluster}.png'),
        alpha=1,
        cmap=cmap_sub,
        column=cluster,
        figsize=(17.5, 11.25)
    )
    
    sdata.table.obs = sdata.table.obs.drop([cluster], axis=1)

In [ ]:
sdata